In [14]:
import pandas as pd
import numpy as np
from pandas import DataFrame
from pandas import Series
from datetime import datetime
from datetime import date
from copy import deepcopy
import json

# Import dataset
aurora_raw = pd.read_csv("customers (3).csv")

In [2]:
aurora_raw.head()
aurora_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 313 entries, 0 to 312
Columns: 219 entries, @01_lactante to @zona_pais
dtypes: bool(2), float64(48), int64(2), object(167)
memory usage: 531.4+ KB


In [15]:
# Rename variables according to Carto names (first round)

newColumns = {
    '@phone': 'phone',
    '@id': 'id',
    '@conexion_date': 'conexion_date',
    '@date_registred': 'date_registered',
    '@0_flow': 'flow_type',
    '@1_gestor': 'gestor',
    '@zona_pais': 'zona_pais'
}

# there are still missing variables of dates, geographical coordinates, feeback of services.
aurora_raw = aurora_raw.rename(columns=newColumns)

In [16]:
# Eliminate observations of Aurora team phones.
aurora_raw = aurora_raw.drop(aurora_raw[aurora_raw.phone == 56994154407].index)
aurora_raw = aurora_raw.drop(aurora_raw[aurora_raw.phone == 56963085715].index)
aurora_raw = aurora_raw.drop(aurora_raw[aurora_raw.phone == 56930150973].index)
aurora_raw = aurora_raw.drop(aurora_raw[aurora_raw.phone == 56956195946].index)
aurora_raw = aurora_raw.drop(
    aurora_raw[aurora_raw.phone == 573245502367].index)
aurora_raw = aurora_raw.drop(
    aurora_raw[aurora_raw.phone == 573002983157].index)
aurora_raw = aurora_raw.drop(
    aurora_raw[aurora_raw.phone == 573044640378].index)
aurora_raw = aurora_raw.drop(
    aurora_raw[aurora_raw.phone == 573133910738].index)
aurora_raw = aurora_raw.drop(
    aurora_raw[aurora_raw.phone == 573185891917].index)
aurora_raw = aurora_raw.drop(
    aurora_raw[aurora_raw.phone == 573182396683].index)
aurora_raw = aurora_raw.drop(
    aurora_raw[aurora_raw.phone == 573206646548].index)
aurora_raw = aurora_raw.drop(
    aurora_raw[aurora_raw.phone == 573216903526].index)
aurora_raw = aurora_raw.drop(
    aurora_raw[aurora_raw.phone == 573105523263].index)
aurora_raw = aurora_raw.drop(aurora_raw[aurora_raw.phone == 50685075883].index)
aurora_raw = aurora_raw.drop(aurora_raw[aurora_raw.phone == 50660030436].index)
aurora_raw = aurora_raw.drop(aurora_raw[aurora_raw.phone == 50763068014].index)
aurora_raw = aurora_raw.drop(aurora_raw[aurora_raw.phone == 50768244084].index)
aurora_raw = aurora_raw.drop(aurora_raw[aurora_raw.phone == 51963852497].index)
aurora_raw = aurora_raw.drop(aurora_raw[aurora_raw.phone == 50768689439].index)
print(aurora_raw.date_registered.info())

<class 'pandas.core.series.Series'>
Index: 294 entries, 0 to 311
Series name: date_registered
Non-Null Count  Dtype 
--------------  ----- 
294 non-null    object
dtypes: object(1)
memory usage: 4.6+ KB
None


In [17]:
# Replace Aurora v2.0 for Enganche, is the same category
aurora_raw['flow_type'] = aurora_raw['flow_type'].replace(to_replace="Aurora v2.0",
                                                          value="Enganche")
# Replace Paso canoas lower case, is the same category
aurora_raw['zona_pais'] = aurora_raw['zona_pais'].replace(to_replace="Paso canoas - Costa Rica",
                                                          value="Paso Canoas - Costa Rica")
aurora_raw['zona_pais'] = aurora_raw['zona_pais'].replace(to_replace="San Juan de Pasto - Colombia",
                                                          value="Ipiales - Colombia")
aurora_raw['zona_pais'] = aurora_raw['zona_pais'].replace(to_replace="los patios - Colombia",
                                                          value="Los patios - Colombia")
aurora_raw['zona_pais'] = aurora_raw['zona_pais'].replace(to_replace="Colchane - Bolivia",
                                                          value="Colchane - Chile")
aurora_raw['zona_pais'] = aurora_raw['zona_pais'].replace(to_replace="Tacna - Perú",
                                                          value="Arica - Chile")


# Replace bad categories in gestor
# aurora_raw['gestor']=aurora_raw['gestor'].replace(to_replace="Viviana Castro",
#           value="9")
# aurora_raw['gestor']=aurora_raw['gestor'].replace(to_replace="Rosa Casanova",
#           value="7")
# aurora_raw['gestor']=aurora_raw['gestor'].replace(to_replace="Deissy Rodríguez",
#           value="10")
# aurora_raw['gestor']=aurora_raw['gestor'].replace(to_replace="Luis David Calderón",
#           value="8")
# aurora_raw['gestor']=aurora_raw['gestor'].replace(to_replace="José Luis Gaete",
#           value="3")
# aurora_raw['gestor']=aurora_raw['gestor'].replace(to_replace="Glenis Barragán",
#           value="6")
aurora_raw['gestor'] = aurora_raw['gestor'].replace(to_replace="El 5",
                                                    value="5")

In [18]:
aurora_raw["date_registered"] = pd.to_datetime(
    aurora_raw["date_registered"], format='%d-%m-%Y %H:%M:%S')
aurora_raw["date_"] = aurora_raw["date_registered"].dt.strftime("%Y-%m-%d")

In [21]:
# Use dictionary to give other values to the variable
gestor_dict = {
    1: "Glenis Barragán",
    2: "José Herrera",
    3: "José Luis Gaete",
    4: "Luis David Calderón",
    5: "Luisa Solis",
    6: "Marianela Varela",
    7: "Massiel Rodriguez",
    8: "Rosa Casanova",
    9: "Sheila Muñoz",
    10: "Valentina Medina",
    11: "Viviana Castro",
    12: "Yuliana Chiquillo",
}


def gestorMap(value):
    try:

        if (type(value) == int):
            return gestor_dict[value]
        if (type(value) == str):
            return gestor_dict[int(value)]
    except Exception as e:
        return value


aurora_raw['gestor'] = aurora_raw['gestor'].apply(lambda x: gestorMap(x))

In [241]:
unpivoted = aurora_raw.groupby(['zona_pais', 'flow_type', 'date_'])[
    'id'].count().reset_index()
pivoted = unpivoted.pivot(
    columns='date_',
    index=['zona_pais', 'flow_type'],
    values='id')
# Reset the index of the pivoted DataFrame
pivoted = pivoted.reset_index()
pivoted

date_,zona_pais,flow_type,2023-10-04,2023-10-11,2023-10-12,2023-10-13,2023-10-14,2023-10-16,2023-10-17,2023-10-18,2023-10-19,2023-10-20,2023-10-21,2023-10-23,2023-10-24
0,Arica - Chile,Enganche,1.0,3.0,NaN,5.0,2.0,2.0,1.0,4.0,NaN,6.0,NaN,3.0,1.0
1,Arica - Chile,Monitoreo-Feedback,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,1.0,NaN,NaN,NaN
2,Arica - Chile,Monitoreo-NNA,NaN,NaN,NaN,NaN,NaN,1.0,NaN,2.0,NaN,NaN,NaN,NaN,NaN
3,Arica - Chile,Monitoreo-Percepción,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN,2.0,NaN,NaN,NaN
4,Arica - Chile,Monitoreo-vocación,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN
5,Arica - Chile,QR-Enganche,NaN,NaN,NaN,2.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
6,Bochalema - Colombia,Enganche,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,3.0
7,Colchane - Chile,Enganche,NaN,NaN,NaN,NaN,NaN,NaN,9.0,7.0,5.0,3.0,NaN,NaN,NaN
8,Colchane - Chile,Monitoreo-Feedback,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN
9,Colchane - Chile,Monitoreo-Percepción,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN


In [274]:
unpivoted = aurora_raw.groupby(['zona_pais', 'gestor', 'date_'])[
    'id'].count().reset_index()
pivoted1 = unpivoted.pivot(
    columns='date_',
    index=['zona_pais', 'gestor'],
    values='id')
# Reset the index of the pivoted DataFrame
pivoted1 = pivoted1.reset_index()
pivoted1

date_,zona_pais,gestor,2023-10-11,2023-10-12,2023-10-13,2023-10-14,2023-10-16,2023-10-17,2023-10-18,2023-10-19,2023-10-20,2023-10-21,2023-10-23,2023-10-24
0,Arica - Chile,10,NaN,NaN,NaN,NaN,2.0,1.0,2.0,NaN,3.0,NaN,1.0,NaN
1,Arica - Chile,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN
2,Arica - Chile,9,NaN,NaN,NaN,NaN,2.0,NaN,8.0,NaN,6.0,NaN,2.0,NaN
3,Arica - Chile,Deissy Rodríguez,2.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Arica - Chile,Luis David Calderón,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Arica - Chile,Luisa Solis,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
6,Arica - Chile,Rosa Casanova,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Arica - Chile,Viviana Castro,1.0,NaN,3.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Bochalema - Colombia,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0
9,Bochalema - Colombia,Luis David Calderón,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


In [275]:
excel_file = 'enganches_gestor.xlsx'
pivoted1.to_excel(excel_file, index=False)